In [1]:
import cv2
from pathlib import Path
from random import *
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage.feature import hog
from imutils import face_utils
import dlib
import os
import pickle
np.random.seed(1000)


In [24]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
frames = []
labels = []
for file in os.listdir('output/'):
    if file[-10:] == 'frames.pkl':
        with open('output/'+file, 'rb') as f:
            frames.append(pickle.load(f))
    elif file[-10:] == 'labels.pkl':
        with open('output/'+file, 'rb') as f:
            labels.append(pickle.load(f))

            
# for file in os.listdir('G:/pkl'):
#     if file[-10:] == 'frames.pkl':
#         with open('G:/pkl'+file, 'rb') as f:
#             frames=frames+pickle.load(f)
#     elif file[-10:] == 'labels.pkl':
#         with open('G:/pkl'+file, 'rb') as f:
#             labels=labels+pickle.load(f)            

# frames = np.asarray(frames)
# labels = np.asarray(labels)
# print(frames.shape)

In [4]:
print(len(frames), len(labels))

9067 9067


In [5]:
from sklearn.model_selection import train_test_split
train_clips, test_clips, train_clips_labels, test_clips_labels = \
    train_test_split(frames, labels, test_size=0.2, random_state=42)

In [6]:
train_images, test_images, train_labels, test_labels = [], [], [], []

for clip, label in zip(train_clips, train_clips_labels):
    try:
        train_images, train_labels = train_images + clip, train_labels + [label[0]] * len(clip)
    except:
        continue

for clip, label in zip(test_clips, test_clips_labels):
    try:
        test_images, test_labels = test_images + clip, test_labels + [label[0]] * len(clip)
    except:
        continue
    
print(len(train_images), len(train_labels), len(test_images), len(test_labels))

1944475 1944475 482212 482212


In [7]:
train_images_seq, test_images_seq, train_labels_seq, test_labels_seq = [], [], [], []

for clip, label in zip(train_clips, train_clips_labels):
    if len(label) and len(clip) == 300:
        train_labels_seq = train_labels_seq + [label[0]]
        train_images_seq.append(clip)


for clip, label in zip(test_clips, test_clips_labels):
    if len(label) and len(clip) == 300:
        test_labels_seq = test_labels_seq + [label[0]]
        test_images_seq.append(clip)
    
print(len(train_labels_seq), len(train_labels_seq), len(test_images_seq), len(test_labels_seq))

6455 6455 1602 1602


In [8]:
train_images_seq, test_images_seq, train_labels_seq, test_labels_seq = np.asarray(train_images_seq, dtype=np.float16), np.asarray(test_images_seq, dtype=np.float16), np.asarray(train_labels_seq, dtype=np.float16), np.asarray(test_labels_seq, dtype=np.float16)

In [9]:
print(train_images_seq.shape, train_labels_seq.shape, test_images_seq.shape, test_labels_seq.shape)

(6455, 300, 48, 48) (6455,) (1602, 300, 48, 48) (1602,)


In [10]:
train_images_seq = np.expand_dims(train_images_seq, axis=4)
test_images_seq = np.expand_dims(test_images_seq, axis=4)

In [11]:
print(train_images_seq.shape, train_labels_seq.shape, test_images_seq.shape, test_labels_seq.shape)

(6455, 300, 48, 48, 1) (6455,) (1602, 300, 48, 48, 1) (1602,)


cnn_input = keras.layers.Input(batch_shape=(None, None, 224, 224, 3), dtype='float32', name='images')
cnn = keras.layers.ConvLSTM2D(filters=192, kernel_size=(3, 3), padding='valid', activation='relu', return_sequences=True)(cnn_input)
cnn = keras.layers.BatchNormalization(axis=-1)(cnn)
cnn = keras.layers.ConvLSTM2D(filters=192, kernel_size=(1, 1), padding='valid', activation='relu', return_sequences=True)(cnn)
cnn = keras.layers.ConvLSTM2D(filters=192, kernel_size=(3, 3), strides = (2, 2), padding='valid', activation='relu', return_sequences=True)(cnn)
cnn = keras.layers.BatchNormalization(axis=-1)(cnn)
cnn = keras.layers.Dropout(0.5)(cnn)
cnn = keras.layers.ConvLSTM2D(filters=96, kernel_size=(3, 3), padding='valid', activation='relu', return_sequences=True)(cnn)
cnn = keras.layers.BatchNormalization(axis=-1)(cnn)
cnn = keras.layers.ConvLSTM2D(filters=96, kernel_size=(1, 1), padding='valid', activation='relu', return_sequences=True)(cnn)
cnn = keras.layers.ConvLSTM2D(filters=96, kernel_size=(1, 1), padding='valid', activation='relu', return_sequences=True)(cnn)
cnn = keras.layers.MaxPooling3D(pool_size=(3, 3, 3), strides=2, data_format="channels_last")(cnn)
cnn = keras.layers.Dropout(0.5)(cnn)
cnn = keras.layers.ConvLSTM2D(filters=32, kernel_size=(3, 3), padding='valid', activation='relu', return_sequences=True)(cnn)
cnn = keras.layers.BatchNormalization(axis=-1)(cnn)
cnn = keras.layers.ConvLSTM2D(filters=32, kernel_size=(3, 3), padding='valid', activation='relu', return_sequences=True)(cnn)
cnn = keras.layers.BatchNormalization(axis=-1)(cnn)
cnn = keras.layers.ConvLSTM2D(filters=32, kernel_size=(3, 3), padding='valid', activation='relu', return_sequences=True)(cnn)
cnn = keras.layers.BatchNormalization(axis=-1)(cnn)
cnn = keras.layers.ConvLSTM2D(filters=3, kernel_size=(1, 1), padding='valid', activation='relu', return_sequences=True)(cnn)
cnn = keras.layers.GlobalAveragePooling3D()(cnn)
cnn = keras.layers.Dense(3, activation='softmax')(cnn)

In [12]:
train_images, test_images, train_labels, test_labels = np.asarray(train_images), np.asarray(test_images), np.asarray(train_labels), np.asarray(test_labels)

In [13]:
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

(1944475, 48, 48) (1944475,) (482212, 48, 48) (482212,)


In [14]:
test_images = np.expand_dims(test_images, axis=3)
train_images = np.expand_dims(train_images, axis=3)

In [15]:
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

(1944475, 48, 48, 1) (1944475,) (482212, 48, 48, 1) (482212,)


In [16]:
test_images1 = np.expand_dims(test_images, axis=3)
train_images1 = np.expand_dims(train_images, axis=3)

model = keras.models.Model(inputs=cnn_input, outputs=cnn)

In [17]:
strain_images, strain_labels, stest_images, stest_labels = train_images[:97223], train_labels[:97223], test_images[:24110], test_labels[:24110]

In [18]:
print(type(train_images_seq[0][0][0][0][0]))

<class 'numpy.float16'>


In [19]:
from tensorflow.keras.layers import TimeDistributed
model = keras.Sequential([
    TimeDistributed(keras.layers.Conv2D(filters=32, input_shape=(48,48,1), kernel_size=3, padding='valid', activation='elu')),
#     keras.layers.BatchNormalization(axis=-1),
    TimeDistributed(keras.layers.Conv2D(filters=64, kernel_size=1, padding='valid', activation='elu')),
    TimeDistributed(keras.layers.Flatten()),
    keras.layers.LSTM(1024,activation = 'elu', return_sequences = True),
    keras.layers.LSTM(512,activation = 'elu'),
    keras.layers.Dense(4, activation='softmax')
])

In [20]:
model.compile(optimizer='adamax',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
model.fit(train_images_seq, epochs=3, batch_size=1)

Train on 6455 samples
Epoch 1/3
   1/6455 [..............................] - ETA: 17:59:36

ResourceExhaustedError: OOM when allocating tensor with shape[135424,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

In [ ]:
test_loss, test_acc = model.evaluate(test_images_seq,  test_labels)